In [1]:
import pandas as pd
import numpy as np
import missingno as msno
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns',100)
pd.set_option('display.max_rows',500)
pd.set_option('display.max_colwidth',-1)

from IPython.core.display import display, HTML
display(HTML('<style>.container { width:100% !important; }</style>'))

### Load Dataset

In [3]:
train_df = pd.read_json('data/seniority.train', lines=True)
test_df = pd.read_json('data/seniority.test', lines=True)
train_df.shape, test_df.shape

((29969, 3), (29999, 3))

In [4]:
df = pd.concat([train_df, test_df],axis=0)

### Text preprocessing

In [5]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer


def convert_lower_case(data):
    return np.char.lower(data)


def remove_punctuation(data):
    symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
    for i in range(len(symbols)):
        data = np.char.replace(data, symbols[i], ' ')
        data = np.char.replace(data, "  ", " ")
    data = np.char.replace(data, ',', '')
    return data


def remove_apostrophe(data):
    return np.char.replace(data, "'", "")


def preprocess(data):
    data = convert_lower_case(data)
    data = remove_punctuation(data) 
    data = remove_apostrophe(data)
    return str(data)

In [6]:
preprocess('B.Tech')

'b tech'

###  Education preprocessing

In [53]:
DEGREES = ['b', 'm', 'a', 'c', 'd', 'p', 's', 'h', 'e', 'g', 'i', 'f', 't', 'l', 'n']

def get_last_education_time(educations):
    edu_times = []
    
    for edu in educations:
        edu_times += edu.get('time')
    
    edu_times = [et for et in edu_times if et!='notKnown']
    latest_education_time = max(edu_times, default=None)
    
    return latest_education_time


def get_highest_degree_level(educations):
    degrees = set()
    
    for edu in educations:
        degrees.add(preprocess(edu.get('degree')[:1]))
    
    return ''.join(sorted(degrees))

def get_proxy_for_degree(educations):
    degree_proxy = []
    
    degrees = get_highest_degree_level(educations)
    return [(deg in degrees) for deg in DEGREES]
        

def time_since_last_education(educations):
    try:    
        
        stime = get_last_education_time(educations)
        t1 = parser.parse(stime)

        s2 = datetime.today().strftime("%Y-%m-%d")
        t2 = parser.parse(s2)
        timedelta = t2 - t1
        return timedelta.days
    except Exception as e:
        # For now catching all exceptions
        # Before sending to production catch specific exception
        return None

### Experience Preprocessing

In [36]:
from datetime import datetime
from dateutil import parser


def get_time_difference(s1,s2):
    
    t1 = parser.parse(s1)
    
    if s2 in ['Present', 'current', 'Current', 'Till Date']:
        s2 = datetime.today().strftime("%Y-%m-%d")
    
    t2 = parser.parse(s2)
    timedelta = t2 - t1
    return timedelta
 

def get_longest_tenure(experiences):
    exp_times = []
    
    for exp in experiences:
        try:
            t1,t2 = exp.get('time')
            timedelta = get_time_difference(t1,t2)
            exp_times.append(timedelta.days)
        except Exception as e:
            print(e)
            
    return max(exp_times,default=0),sum(exp_times)

### Preprocess profile

In [51]:
def process_profile(profile):
    processed_profile = []
    
    educations = profile.get('education')
    experiences = profile.get('experience')
    
    #Number of educations
    processed_profile.append(len(educations))
    
    #Experience count
    processed_profile.append(len(experiences))
    
    # Work tenure info
    longest_tenure,total_work_ex = get_longest_tenure(experiences)
    processed_profile += [longest_tenure,total_work_ex]
    
    # Using proxy for degrees he has mentioned
    processed_profile += get_proxy_for_degree(educations)
    
    # Days since last education
    processed_profile += [time_since_last_education(educations)]
    
    return processed_profile

### Load Random Forest Model

In [29]:
from joblib import dump, load
clf = load('models/seniority_rfr.joblib')
# clf.predict([list(example.values())])

In [32]:
profile = {"seniority_level": "Mid-Level", "education": [{"school": "University of Wisconsin Madison", "description": "University of Wisconsin - Madison   Madison, WI\n\nSeptember 2005 - December 2009\nBachelor of Business Administration, Marketing, GPA 3.5/4.0", "degree": "Bachelor of Business Administration", "time": ["2005-09-01", "2009-12-01"]}, {"school": "UW", "description": "Certificate in Spanish Studies\nDean's List - Fall 2005\nUW Real Estate Club (4 Semesters)", "degree": "Certificate in Spanish", "time": ["notKnown", "2005-09-01"]}, {"school": "Universidad de Sevilla", "description": "Business Action for Sustainable Enterprise, B.A.S.E. (6 Semesters)\n\nUniversidad de Sevilla   Sevilla, Spain\nApril 2007 - December 2007", "degree": "B.A.S.E.", "time": ["2007-04-01", "2007-12-01"]}, {"school": "American TESOL Institute", "description": "American TESOL Institute   Phuket, Thailand\nApril 2010\nIntense 3-week teaching course\nInternationally recognized TEFL certificate, with a practical emphasis on teaching skills, cultural training, language awareness, phonology and classroom management", "degree": "certificate", "time": ["notKnown", "2010-04-01"]}], "experience": [{"skills": ["marketing", "campaigns", "management", "marketing automation", "roi", "salesforce.com", "direct mail", "analytics", "analysis", "strategy", "metrics", "campaign effectiveness", "webinars", "creative development", "online", "selection", "foundation", "dinners", "engagement", "lead scoring"], "time": ["2014-03-01", "Present"], "work": "Quantum Secure", "description": "\u2022 Optimized and managed Salesforce.com instance to create a useful and productive lead and opportunity management tool\n\u2022 Designed and executed various demand generation campaigns including webinars, tradeshows, roundtable dinners, online, direct mail and email\n\u2022 Implemented advanced marketing program analytics to track metrics, including campaign effectiveness, prospect engagement, funnel advancement, conversion rate and ROI to identify successes opportunities for improvement\n\u2022 Developed custom nurture tracks for prospects and existing customers designed to educate and engage them with relevant content as they move through the funnel\n\u2022 Lead efforts to build overall marketing automation strategy and foundation, including campaign flow templates, nurture paths, lead scoring, lead stages and best practices for the team to adopt\n\u2022 Continuously monitored and analyzed audience selection, timing, messaging, testing, creative development, and analysis of all campaigns", "title": "Marketing Program Manager"}, {"skills": ["marketing", "campaigns", "sales", "social media", "outbound", "management", "budget", "lead generation", "marketing automation", "email marketing", "seo", "marketing campaigns", "logistics", "contract negotiations", "marketing initiatives", "email campaigns", "promotional", "marketing promotions", "communications", "sales tools"], "time": ["2013-01-01", "2014-02-28"], "work": "Gilson, Inc", "description": "\u2022 Conception, development and execution of demand and lead generation marketing campaigns including direct and email marketing, promotions, landing pages, SEO, and support materials that align with business initiatives\n\u2022 Manage all aspects of trade show events (attendees ranging from 150-30,000 per event) including structure logistics, contract negotiations, staffing, promotional giveaways, graphics, pre/post-show marketing, show objectives, etc., often managing multiple shows at the same time; consistently came in under budget\n\u2022 Monitor and analyze effectiveness of all communications activities and outbound programs, including A/B testing for email campaigns, web content and social media involvement\n\u2022 Implementation and management of cloud applications/tools to support sales & marketing initiatives, including marketing automation and demand gen tools (Marketo, Google Adwords), and sales quoting tools improvement\n\u2022 Writing, editing and designing of inbound and outbound marketing content including emails, web and landing pages, brochures, social media, etc. - part of multi-touch creative campaigns", "title": "Senior Marketing Specialist"}, {"skills": ["marketing", "sales", "social media", "budget", "campaigns", "media placement", "management", "advertising", "lead generation", "crm", "sales team", "roi", "digital advertising", "product positioning", "analysis", "marketing budget", "printers", "newsletters", "hygiene", "coordination"], "time": ["2011-05-01", "2013-01-01"], "work": "", "description": "\u2022 Actively maintain consistent product positioning and accurate information across all public mediums (websites, newsletters, social media, print publications, etc.)\n\u2022 Support of sales team of 30+ across the country at any stage of the sales process\n\u2022 Assistance with the maintenance of company-wide CRM, improving upon best practices and ensuring good data hygiene for best campaign performance\n\u2022 Improvement of monthly analysis of key marketing initiatives to assess the performance of marketing campaigns in order to gain insight on ROI\n\u2022 Management of media/advertising budget of over $500K and Marketing budget exceeding $1M; dictating all media placement/design and leading company transition into digital advertising\n\u2022 Integral part of lead generation improvement of over 900% through improved media placement and social media\n\u2022 Coordination of marketing efforts with external suppliers (graphic/marketing agencies, printers, mail-houses, etc.)", "title": "Marketing Specialist"}, {"skills": ["classroom", "negotiation", "communication", "community", "courses", "flexibility", "thai", "events", "independence"], "time": ["2010-05-01", "2011-03-01"], "work": "Supaluck School", "description": "\u2022 Set up classroom meetings and events to help parents become more involved and build rapport within community\n\u2022 Learned conversational Thai and honed in on my verbal and non-verbal communication skills\n\u2022 Lead courses focusing on Reading Comprehension, Listening Skills, Writing and Speaking\n\u2022 Increased flexibility, independence, and negotiation skills", "title": "Intensive ESL Teacher"}, {"skills": ["marketing", "sales", "advertising", "social media", "management", "budget", "crm", "sales team", "direct marketing", "media plan", "facebook", "database", "collateral", "networking", "sales meetings", "research", "green", "clients", "features", "house"], "time": ["2008-05-01", "2010-04-30"], "work": "T. Wall Properties", "description": "\u2022 Supported Sales & Marketing team through creation of sales collateral, advertising, direct marketing and lease proposal booklets\n\u2022 Completed a strong, visible social media plan using Twitter, LinkedIn, Facebook, CEO blog, etc.\n\u2022 Managed complex prospect database/CRM system and implemented new procedures making it a more valuable research tool company wide\n\u2022 Composed new layout for company website to include a green link in order to educate potential clients of the company's green features; tracked web page traffic; responsible for website updates\n\u2022 Planned, budgeted, and hosted large corporate events including annual investor meetings (over 250 people), broker networking functions, and ground breaking/open house events\n\u2022 Acquired, analyzed and interpreted consumer and market information to optimize value for brokers\n\u2022 Assisted in creation and management of department budget of over $120,000 positioning advertising/marketing spending\n\u2022 Collaborated closely with Sales team; led weekly sales meetings to evaluate quarterly and individual goals\n\u2022 Contributed to the response to and research for RFP's & RFQ's", "title": "Marketing Intern/Assistant and Sales and Leasing Coordinator"}]}

In [48]:
pp = process_profile(profile)

In [52]:
pp

[4,
 5,
 2024,
 4091,
 True,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 3454]

In [50]:
clf.predict([pp])

array(['Mid-Level'], dtype=object)

### Improvements
#### - add more comments
#### - add proper logging before sending to production, Use logger
#### - in logger also log which version of model is running (store model_id)
#### - Normalizing the data, data enrichment for both degrees and titles
#### - Using better clusters for both degrees and titles
#### - Add classes
#### - Put code in proper folders
##### models, parsers, model_training_code, requirements.txt, README.md
#### - Keep loggin production results(query,predicted) , so that we can keep track of performance.

#### - Test Cases for parsers(eduction_parser, experience_parser)
#### - Use title and industry to better understand seniority, For e.g. Google VP is very se
#### - Measure of performance should include deviation from true label, for e.g. manager predicted as director is not very wrong but director predicted as intern is verry wrong